## Importing Required Libraries

In [56]:
import keras
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications.vgg19 import VGG19
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from keras import backend as K
import cv2
import numpy as np
from tqdm import tqdm
import os
import matplotlib
from pathlib import Path
import random
from PIL import Image

## Setting Required Parameters

In [57]:
# Set this to true if you want to create Rectangle and Circle Images with different textures
generate_rect_circle_data = False

# Below parameters are for creating Synthetic Images which has a foreground fiber texture and background gray texture.
generate_texture_data = True
add_border = False
adding_noise = True
num_black_pixels = 50000 # based on the required image size 500 x 500.
num_images = 16135
border_width = 5
gray_image = False

In [4]:
def load_texture_image(path):
    texture_image = cv2.imread(path)
    texture_image = cv2.cvtColor(texture_image, cv2.COLOR_BGR2RGB)
    return texture_image

In [5]:

'''
This function loads the texture image and the desired shape and generates a synthetic image. The desired shape is either a circle or a rectangle.
if circle is True, a random center point on the syntethic image and a random radius greater than 50 pixels is generated. Then everywhere except the circle is blacked out.
if circle is False, a random top-left corner point on the syntethic image and a random width and height greater than 50 pixels is generated. Then everywhere except the rectangle is blacked out.
'''
def generate_synthetic_image(texture_image, circle=True):
    # Generate a random center point and radius for the circle
    if circle:
        center_x = np.random.randint(100, texture_image.shape[1] - 100)
        center_y = np.random.randint(100, texture_image.shape[0] - 100)
        radius = np.random.randint(75, texture_image.shape[0] - center_y)
        # Create a black image
        synthetic_image = np.zeros(texture_image.shape, dtype=np.uint8)
        # Draw a circle on the black image
        cv2.circle(synthetic_image, (center_x, center_y), radius, (255, 255, 255), -1)
        # Black out everything except the circle
        synthetic_image = cv2.bitwise_and(texture_image, synthetic_image)
    # Generate a random top-left corner point and width and height for the rectangle
    else:
        top_left_x = np.random.randint(0, texture_image.shape[1] - 100)
        top_left_y = np.random.randint(0, texture_image.shape[0] - 100)
        width = np.random.randint(100, texture_image.shape[1] - top_left_x)
        height = np.random.randint(100, texture_image.shape[0] - top_left_y)
        # Create a black image
        synthetic_image = np.zeros(texture_image.shape, dtype=np.uint8)
        # Draw a rectangle on the black image
        cv2.rectangle(synthetic_image, (top_left_x, top_left_y), (top_left_x + width, top_left_y + height), (255, 255, 255), -1)
        # Black out everything except the rectangle
        synthetic_image = cv2.bitwise_and(texture_image, synthetic_image)
    return synthetic_image


In [6]:

# Get the path to the directory the where the images are going to be saved and the number of images to be generated
def generate_images(texture_image, save_dir, num_images, circle=True):
    # Create the directory if it does not exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for i in range(num_images):
        synthetic_image = generate_synthetic_image(texture_image, circle)
        ret = cv2.imwrite(os.path.join(save_dir, 'synthetic_image_' + str(i) + '.jpg'), synthetic_image)

In [7]:
# This function gives a random value based on a distribution.
def weighted_randint(low, high):
    """
    Generate a random integer between low and high with equal probability for values in the range,
    and lower probability for values outside the range.
    """
    weights = [1] * (high - low + 1)
    weights[0] = weights[-1] = 0.1  # Lower probability for values at the ends of the range
    weights_sum = sum(weights)
    weights = [w / weights_sum for w in weights]
    return random.choices(range(low, high+1), weights=weights)[0]


In [8]:
# This function adds noise to the image
def add_noise(img, num_black_pixels):
    # Convert the image to grayscale if needed
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Add Gaussian noise to the image
    mean = 0
    variance = 5
    sigma = variance ** 0.5
    gaussian = np.random.normal(mean, sigma, img.shape)
    noisy_image = img + gaussian.astype(np.uint8)
    noisy_image = keras.utils.array_to_img(noisy_image)
    
    noisy_image = add_black_pixels(img, num_black_pixels)
    return noisy_image

In [9]:
# This function adds black pixels to the noise image to reduce the noise.
def add_black_pixels(img, num_black_pixels):

    # Set a random location for each black pixel
    h, w, _ = img.shape
    for i in range(num_black_pixels):
        x, y = np.random.randint(0, w), np.random.randint(0, h)
        img[y, x] = [0, 0, 0]  # Set the pixel to black

    return img

In [10]:
def write_to_file(file_path, line):
    with open(file_path, 'a') as file:
        file.write(line + '\n')

In [11]:
def clear_contents(file_path):
    with open(file_path, 'w') as file:
        pass

In [17]:
def gray_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Create three copies of the grayscale image
    red = gray.copy()
    green = gray.copy()
    blue = gray.copy()

    # Create new numpy arrays for the channel images
    red_channel = np.zeros_like(gray)
    green_channel = np.zeros_like(gray)
    blue_channel = np.zeros_like(gray)

    # Set pixel values for each channel image
    red_channel[:, :] = red
    green_channel[:, :] = green
    blue_channel[:, :] = blue

    # Merge the three channel images into a single color image
    gray_img = cv2.merge((blue_channel, green_channel, red_channel))
    return gray_img
    

In [24]:

gray_image_ind = True
counter = 0

if generate_texture_data:
    
    #clearing the contents of the text file before starting
#     clear_contents(r'C:\Users\AVLguest\work\Random_experiments\Synthetic_texture\new_data/texture_4.txt')
    
    #output image resolution
    bottom_image_width = 500
    bottom_image_height = 500

    # Load the texture images
    texture_img = cv2.imread(r"C:\Users\AVLguest\work\Random_experiments\Synthetic_texture\new_data/wood_4.png")
    bottom_image = Image.open(r"C:\Users\AVLguest\work\Random_experiments\Synthetic_texture\new_data/light-gray.jpg")
    border_image = cv2.imread(r"C:\Users\AVLguest\work\Random_experiments\Synthetic_texture\new_data/gray.png")

    #loading the background texture, resizing it and converting back to image format.
    bottom_image = keras.utils.img_to_array(bottom_image)
    bottom_image = cv2.resize(bottom_image, (bottom_image_width,bottom_image_height), )
    bottom_image = keras.utils.array_to_img(bottom_image)

    # Define the image which is used for fiber creation, kept larger than output image to create edge fibers.
    new_img_width = 700
    new_img_height = 700

    # resizing the texture image which is used for fiber creation.
    texture_img = cv2.resize(texture_img, (new_img_width, new_img_height))
    texture_img = cv2.cvtColor(texture_img, cv2.COLOR_BGR2RGB)


    #displaying the texture image for fiber.
#     plt.imshow(texture_img)
#     plt.show()

    # Iterations through number of fiber images required.
    for j in tqdm(range(0,num_images)):

        # randomly choosing between 1-3 fibers
        no_of_fibers = random.choices([1,2,3], weights = [1,2,3])[0]

        #randomly defining the width of the fiber.
        size_of_fiber = random.randint(30,70)

        # creating a black canvas where the created fibers are pasted.
        fiber_image = np.zeros((new_img_height, new_img_width, 3), dtype=np.uint8)
        
        foreground_pixels = 0
        # Iterating through number of fibers required in that particular image.
        for count in range(no_of_fibers):

            # randomly selecting the start location of fiber that is created vertically.started from size of the fiber to avaoid the breaking of fiber.
            strip_x = random.choice([size_of_fiber,weighted_randint(0,new_img_width)])
                
            #if the location is 0 then we make the fiber horizantal and randomly select a position in the y-axis
            if strip_x == 0 :
                mode = "horizontal"
                strip_y = random.choice([size_of_fiber,weighted_randint(0,new_img_height)])
            else:
                mode = "vertical"
                strip_y = 0

            # Defining Width and Height for each mode.(width ranging from 3 to 7)
            if mode == "horizontal": 
                strip_width = random.randint(3,7)
                strip_height = size_of_fiber
            else:
                strip_width = size_of_fiber
                strip_height = random.randint(3,7)

            # creating a black canvas for each fiber creation and copying the fiber to fiber_image variable later to bring all fibers to a single image i.e., fiber_image.
            new_img = np.zeros((new_img_height, new_img_width, 3), dtype=np.uint8)

            #randomly rotating the texture image before creating the fiber
            angle = random.choice([90,270,360])
            img = keras.utils.array_to_img(texture_img)
            img = img.rotate(angle)
            texture_img = keras.utils.img_to_array(img)
            
#             plt.imshow(texture_img)
#             plt.show()

            # creating single fiber which is vertical in the new_img variable
            if mode == "vertical":
                for i in range(int(new_img_height/strip_height)):
                    new_img[strip_y:strip_y + strip_height, strip_x:strip_x + strip_width] = texture_img[strip_y:strip_y + strip_height, strip_x:strip_x + strip_width]
                    if add_border:
                        try:
                            new_img[strip_y:strip_y + strip_height, strip_x - border_width: strip_x] = border_image[strip_y:strip_y + strip_height, strip_x - border_width: strip_x]
                            new_img[strip_y:strip_y + strip_height, strip_x + strip_width: strip_x + strip_width + border_width] = border_image[strip_y:strip_y + strip_height, strip_x + strip_width: strip_x + strip_width + border_width]
                            
                        except ValueError as e:
                            continue
        
                    strip_y = strip_y + strip_height
                    if i%20 == 0:
                        weights = random.choice([[1,6], [6,1]])
                    shift = random.choices([-1, 1], weights=weights)[0]
                    strip_x = strip_x + shift
                    
#                     plt.imshow(new_img)
#                     plt.show()

            # creating single fiber which is horizontal in the new_img variable
            else:   
                for i in range(int(new_img_width/strip_width)):
                    new_img[strip_y:strip_y + strip_height, strip_x:strip_x + strip_width] = texture_img[strip_y:strip_y + strip_height, strip_x:strip_x + strip_width]
                    if add_border:
                        try:
                            new_img[strip_y:strip_y + strip_height, strip_x - border_width: strip_x] = border_image[strip_y:strip_y + strip_height, strip_x - border_width: strip_x]
                            new_img[strip_y:strip_y + strip_height, strip_x + strip_width: strip_x + strip_width + border_width] = border_image[strip_y:strip_y + strip_height, strip_x + strip_width: strip_x + strip_width + border_width]
                            
                        except ValueError as e:
                            continue
                    strip_x = strip_x + strip_width
                    if i%20 == 0:
                        weights = random.choice([[1,6], [6,1]])
                    shift = random.choices([-1, 1], weights=weights)[0]
                    strip_y = strip_y + shift

            # Visualizing each fiber created above seperately on black canvas.
#             plt.imshow(new_img)
#             plt.show()

            # Randomly rotating the fiber image  before pasting in the fiber_image variable.
            img = keras.utils.array_to_img(new_img)
            angle = random.randint(0,360)
            img2 = img.rotate(angle)

            # Creating a new black canvas with dimensions as fiber_image 
            new_image = Image.new('RGB', (new_img_height, new_img_width), (0, 0, 0))

            #converting the fiber_image back to image format.
            fiber_image = keras.utils.array_to_img(fiber_image)

            # adding the fibers to the fiber_image such that the fibers are only added to the black pixels removing overlap
            for x in range(new_img_width):
                for y in range(new_img_height):
                    fiber_pixel = fiber_image.getpixel((x, y))
                    top_pixel = img2.getpixel((x, y))
                    if top_pixel != (0, 0, 0):
                        new_image.putpixel((x, y), top_pixel)
                    else:
                        new_image.putpixel((x, y), fiber_pixel)
            
            # making the new_image as fiber_image to continue adding the fibers in the loop
            fiber_image = new_image


        fiber_image = keras.utils.img_to_array(fiber_image)
        
        if adding_noise:
            fiber_image = add_noise(fiber_image, num_black_pixels)

        # Rotating the top image randomly and cropping it to actual output size.
        top_image = keras.utils.array_to_img(fiber_image)
        angle = random.randint(0,360)
        top_image = top_image.rotate(angle)
        top_image = top_image.crop(((new_img_width - bottom_image_width)//2,(new_img_height - bottom_image_height)//2,new_img_width - ((new_img_width - bottom_image_width)//2),new_img_height - ((new_img_height - bottom_image_height)//2)))
        
        # Rotating the background with a random angle
        angle = random.choice([90,270,360])
        bottom_image = bottom_image.rotate(angle)

        # Create a new image and blend the two images using the non-black pixels of the top image
        new_image = Image.new('RGB', (bottom_image_width,bottom_image_height), (0, 0, 0))

        # Adding the fiber image to the backgroung texture image.
        for x in range(bottom_image_width):
            for y in range(bottom_image_height):
                bottom_pixel = bottom_image.getpixel((x, y))
                top_pixel = top_image.getpixel((x, y))
                if top_pixel != (0, 0, 0):
                    new_image.putpixel((x, y), top_pixel)
                    foreground_pixels += 1
                else:
                    new_image.putpixel((x, y), bottom_pixel)
                    
        foreground_ratio = round((foreground_pixels) / (bottom_image_width * bottom_image_height) ,2)
        background_ratio = round(((bottom_image_width * bottom_image_height) - foreground_pixels) / (bottom_image_width * bottom_image_height), 2)
            
                    
        if gray_image_ind:
            new_image = keras.utils.img_to_array(new_image)
            new_image = gray_image(new_image)
            new_image = keras.utils.array_to_img(new_image)
            
            
#         print(foreground_ratio)
        if foreground_ratio > 0.05:
            counter +=1
            #writing the info into the text file.
            line = "image_" + str(counter) + " - " + "foreground ratio - " + str(foreground_ratio)  + "," + "background ratio - " + str(background_ratio) + ",foreground pixel count - " + str(foreground_pixels) 
            write_to_file(r'C:\Users\AVLguest\work\Random_experiments\Synthetic_texture\new_data/wood_4.txt', line)
            
            #Saving the image.
            new_image.save(r"C:\Users\AVLguest\work\Random_experiments\Synthetic_texture\new_data\wood_4_images/" + "image_" + str(counter) + ".jpg")
        else:
            j = j - 1

 61%|██████    | 9701/16000 [10:10:45<6:36:34,  3.78s/it]


KeyboardInterrupt: 

In [19]:
# after generating the images, a text file is generated which contain the foreground and background ratio of all the images.
# this code calculates the average foreground to background ratio for whole dataset.
def foreground_ratio_calculation(file_path):
    foreground_sum = 0
    line_count = 0

    with open(file_path, "r") as file:
        for line in file:
            foreground_ratio = float(line[line.index("-") + 21: line.index(",")]) 
            foreground_sum += foreground_ratio
            line_count += 1

    if line_count > 0:
        average_foreground_ratio = foreground_sum / line_count
        print("Average Foreground Ratio:", average_foreground_ratio)
    else:
        print("No lines found in the file.")
        
foreground_ratio_calculation(r"C:\Users\AVLguest\work\Random_experiments\Synthetic_texture\new_data/texture_2.txt")
# renamed to texture_2_new.txt
foreground_ratio_calculation(r"C:\Users\AVLguest\work\Random_experiments\Synthetic_texture\new_data/texture_4.txt")


Average Foreground Ratio: 0.13093873687833038
Average Foreground Ratio: 0.1317495063953954


In [48]:
if generate_rect_circle_data:
    texture_image = load_texture_image(r"C:\Users\AVLguest\work\Random_experiments\Synthetic_texture\new_data/metal_texture.jpg")
    generate_images(texture_image, r'C:\Users\AVLguest\work\Random_experiments\Synthetic_texture\new_data/images/', 10, True)
